In [48]:
# Load libraries
import warnings
import numpy as np
import pandas as pd
import os
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score, train_test_split
from sklearn.exceptions import ConvergenceWarning
from sklearn.metrics import classification_report
SEED = 90089

In [16]:
# disable deprecation warnings and convergence warning after production 
os.environ["PYTHONWARNINGS"] = "ignore::UserWarning"

### `LogisticRegression` Classifier on Hyperclygemic patients

In [28]:
# Read data
patient_df = pd.read_csv('../data/hyperglycemic_patients_w_categories.csv', usecols=list(range(3,18)))

# Previewing data
print("Dataframe shape:   ", patient_df.shape)
print("Dataframe Features:", patient_df.columns)
patient_df.head()

Dataframe shape:    (14077, 15)
Dataframe Features: Index(['glucose_max', 'anchor_age', 'dod', 'gender', 'dbp_mean', 'sbp_mean',
       'glucose_mean', 'heart_rate_mean', 'spo2_mean', 'resp_rate_mean',
       'temperature_mean', 'apsiii', 'glucose_score', 'avg_bmi_value',
       'label'],
      dtype='object')


,glucose_max,anchor_age,dod,gender,dbp_mean,sbp_mean,glucose_mean,heart_rate_mean,spo2_mean,resp_rate_mean,temperature_mean,apsiii,glucose_score,avg_bmi_value,label
0,277.0,36,0,0,64.320000,112.880000,221.272727,101.800000,92.000000,17.346154,37.186250,38,3,26.750000,0
1,2340.0,35,0,0,77.769231,122.961538,468.592593,105.361111,96.093023,19.813953,36.727586,87,5,30.558477,0
2,531.0,23,1,0,64.169118,119.661765,275.352941,154.028986,83.698413,25.584615,39.725000,158,5,30.558477,1
3,259.0,19,1,1,49.300000,74.200000,259.000000,89.750000,70.000000,26.333333,31.900000,88,3,30.558477,0
4,406.0,24,0,0,57.333333,92.848485,292.000000,86.631579,96.162162,15.289474,33.590000,150,5,30.558477,1


### LR without `StandardScaler`

In [29]:
# Data split
X = patient_df.loc[:, patient_df.columns != 'label']
y = patient_df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [30]:
# Tune model with stratifiedKFold cross validation
lr = LogisticRegression(random_state=SEED)

parameters = {
    'penalty' : ['l1', 'l2'],
    'C'       : [0.1, 1, 10],
    'solver'  : ['liblinear', 'saga'], 
    'max_iter': [1000, 2000]}
grid_lr = GridSearchCV(lr, parameters, cv=5, n_jobs=8)
grid_lr.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=90089), n_jobs=8,
             param_grid={'C': [0.1, 1, 10], 'max_iter': [1000, 2000],
                         'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'saga']})

In [31]:
print("----         Results from Grid Search           ---")
print("\n The best estimator across ALL searched params:\n", grid_lr.best_estimator_)
print("\n The best parameters across ALL searched params:\n", grid_lr.best_params_)

----         Results from Grid Search           ---

 The best estimator across ALL searched params:
 LogisticRegression(C=1, max_iter=1000, penalty='l1', random_state=90089,
                   solver='liblinear')

 The best parameters across ALL searched params:
 {'C': 1, 'max_iter': 1000, 'penalty': 'l1', 'solver': 'liblinear'}


In [32]:
best_lr = LogisticRegression(C=1,
                            penalty='l1',
                            solver='liblinear',
                            random_state=SEED,
                            max_iter=1000,
                            n_jobs=4)

In [33]:
best_lr.fit(X_train, y_train)

LogisticRegression(C=1, max_iter=1000, n_jobs=4, penalty='l1',
                   random_state=90089, solver='liblinear')

In [34]:
print("Model Accuracy based on Testing dataset: {:.5f}".format(best_lr.score(X_test, y_test)))

Model Accuracy based on Testing dataset: 0.82493


### LR Model with `StandardScaler`

In [36]:
numerical_features = ['glucose_max', 'dbp_mean', 'sbp_mean', 'glucose_mean', 'anchor_age',
                      'heart_rate_mean', 'spo2_mean', 'resp_rate_mean', 'temperature_mean', 
                      'apsiii', 'glucose_score', 'avg_bmi_value']

# Standardized variable for consistent measurement across numerical values
scaler = StandardScaler()
X_std = scaler.fit_transform(patient_df[numerical_features])

# Interpolate categorical variables
X_cat = patient_df[['dod', 'gender']]


X = np.concatenate([X_std, X_cat.to_numpy()], axis=1)
y = patient_df["label"]
print("X Standardized data Shape: ", X_std.shape)
print("X Categorical data shape:  ", X_cat.shape)
print("X, y shape:                ", X.shape, y.shape)

X Standardized data Shape:  (14077, 12)
X Categorical data shape:   (14077, 2)
X, y shape:                 (14077, 14) (14077,)


In [37]:
X_train_std, X_test_std, y_train_std, y_test_std = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [39]:
lr_std = LogisticRegression(random_state=SEED)

parameters = {
    'penalty' : ['l1', 'l2'],
    'C'       : [0.1, 1, 10],
    'solver'  : ['liblinear', 'saga'], 
    'max_iter': [1000, 2000, 3000]}
grid_lr_std = GridSearchCV(lr_std, parameters, cv=5, n_jobs=6)
grid_lr_std.fit(X_train_std, y_train_std)

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=90089), n_jobs=6,
             param_grid={'C': [0.1, 1, 10], 'max_iter': [1000, 2000, 3000],
                         'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'saga']})

In [40]:
print("----         Results from Grid Search           ---")
print("\n The best estimator across ALL searched params:\n", grid_lr_std.best_estimator_)
print("\n The best parameters across ALL searched params:\n", grid_lr_std.best_params_)

----         Results from Grid Search           ---

 The best estimator across ALL searched params:
 LogisticRegression(C=1, max_iter=1000, penalty='l1', random_state=90089,
                   solver='liblinear')

 The best parameters across ALL searched params:
 {'C': 1, 'max_iter': 1000, 'penalty': 'l1', 'solver': 'liblinear'}


In [45]:
best_lr_std = LogisticRegression(C = 0.1,
                                penalty = 'l2',
                                solver = 'saga',
                                random_state=SEED,
                                max_iter=1000, 
                                n_jobs=4)
best_lr_std.fit(X_train_std, y_train_std)

LogisticRegression(C=0.1, max_iter=1000, n_jobs=4, random_state=90089,
                   solver='saga')

In [46]:
print("Model Accuracy based on Testing dataset: {:.5f}".format(best_lr_std.score(X_test_std, y_test_std)))

Model Accuracy based on Testing dataset: 0.82848


In [49]:
y_pred = best_lr_std.predict(X_test_std)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.98      0.84      0.91      2692
           1       0.14      0.48      0.22       123
           2       0.00      0.00      0.00         1

    accuracy                           0.83      2816
   macro avg       0.37      0.44      0.37      2816
weighted avg       0.94      0.83      0.88      2816

